# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps


# Import API key
from api_keys import geoapify_key

geoapify_key = "f76b71d9aa0c4c4d8e5b01cff5e06c63"

print(geoapify_key)

f76b71d9aa0c4c4d8e5b01cff5e06c63


In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barreirinhas,-2.7469,-42.8264,296.59,96,100,0.34,BR,1680835214
1,1,hilo,19.7297,-155.0900,296.62,95,100,4.12,US,1680834953
2,2,imeni babushkina,59.7573,43.1285,267.21,93,6,2.35,RU,1680835214
3,3,klerksdorp,-26.8521,26.6667,287.47,58,0,1.71,ZA,1680835215
4,4,san patricio,28.0170,-97.5169,291.07,95,100,5.58,US,1680835215


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
%%capture --no-display

# Configure the map plot
# Configure gmaps:
gmaps.configure(api_key=geoapify_key)

# Using Lat and Lng as locations:
locations = city_data_df[["Lat", "Lng"]]

# Using Humidity as the weight: 
weights = city_data_df["Humidity"]

# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data
# Zero cloudiness
cleaned_df = city_data_df.loc[(city_data_df['Max Temp']>27)&(city_data_df['Max Temp']>21)&
                            (city_data_df["Wind Speed"]<4.5)&(city_data_df["Cloudiness"]==0)]
cleaned_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,klerksdorp,-26.8521,26.6667,287.47,58,0,1.71,ZA,1680835215
17,17,tando adam,25.7637,68.6610,301.06,18,0,2.47,PK,1680835218
20,20,chitral,35.8462,71.7858,281.94,50,0,3.08,PK,1680835219
24,24,te anau,-45.4167,167.7167,285.00,50,0,0.78,NZ,1680835220
43,43,jamestown,42.0970,-79.2353,280.92,59,0,3.09,US,1680835226


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df=cleaned_df.loc[:,["City_ID", "City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.reset_index()

,index,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
0,3,3,klerksdorp,ZA,-26.8521,26.6667,58,
1,17,17,tando adam,PK,25.7637,68.6610,18,
2,20,20,chitral,PK,35.8462,71.7858,50,
3,24,24,te anau,NZ,-45.4167,167.7167,50,
4,43,43,jamestown,US,42.0970,-79.2353,59,
...,...,...,...,...,...,...,...,...
70,523,523,horsham,GB,51.0640,-0.3272,87,
71,531,531,quimper,FR,48.0000,-4.1000,93,
72,543,543,cabo san lucas,MX,22.8909,-109.9124,53,
73,558,558,fayaoue,NC,-20.6500,166.5333,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":"20",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city_lat=hotel_df.loc[index,"Lat"]
    city_lng=hotel_df.loc[index,"Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_lng},{city_lat},{radius}"
    params["bias"] = f"proximity:{city_lng},{city_lat}"
       
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
klerksdorp - nearest hotel: No hotel found
tando adam - nearest hotel: No hotel found
chitral - nearest hotel: Chitral Inn
te anau - nearest hotel: Distinction Hotel
jamestown - nearest hotel: DoubleTree Jamestown
port elizabeth - nearest hotel: Waterford Hotel
vredefort - nearest hotel: No hotel found
provideniya - nearest hotel: No hotel found
acapulco - nearest hotel: Hotel del Valle
qaanaaq - nearest hotel: Hotel Qaanaaq
hovd - nearest hotel: Khovd hotel
dakar - nearest hotel: 翁记小吃
vicuna - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
richmond heights - nearest hotel: Fairbridge Hotel
black river - nearest hotel: No hotel found
port hueneme - nearest hotel: No hotel found
hurricane - nearest hotel: Wingate by Wyndham Hurricane Near Zion National Park
khalkhal - nearest hotel: خانه معلم
los llanos de aridane - nearest hotel: Valle Aridane
salalah - nearest hotel: Muscat International Hotel
h

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
3,3,klerksdorp,ZA,-26.8521,26.6667,58,No hotel found
17,17,tando adam,PK,25.7637,68.6610,18,No hotel found
20,20,chitral,PK,35.8462,71.7858,50,Chitral Inn
24,24,te anau,NZ,-45.4167,167.7167,50,Distinction Hotel
43,43,jamestown,US,42.0970,-79.2353,59,DoubleTree Jamestown
...,...,...,...,...,...,...,...
523,523,horsham,GB,51.0640,-0.3272,87,Premier Inn
531,531,quimper,FR,48.0000,-4.1000,93,Hôtel Gradlon
543,543,cabo san lucas,MX,22.8909,-109.9124,53,Comfort Rooms
558,558,fayaoue,NC,-20.6500,166.5333,57,Hôtel Beaupré
